### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

sys.path.append("../../data_analysis/")
import helper as hp

In [3]:
savepath = "A03_elise_vs_patent/"
os.makedirs(savepath, exist_ok=True)

In [4]:
df = pd.read_excel(f"A01_patent_peptides_for_ordering/AF2_Predictions_Peptides_Hair_Keratin_1_Coil_2.xlsx")
df.head(2)

,File Name,Peptide Sequence,Peptide Length,Cys Number,Cys Content (# of Cys / Length),Lys Number,Lys Content (# of Lys / Length),PTM Score,IPTM Score,Confidence Score (0.2PTM + 0.8IPTM)
0,Hair_Keratin_1_Coil_2_Pep_Seq_16_scores_rank_0...,KKVELFPK,8,0,0.000000,3,0.375,0.66,0.64,0.644
1,Hair_Keratin_1_Coil_2_Pep_Seq_50_scores_rank_0...,GGVCGPSPPCIT,12,2,0.166667,0,0.000,0.66,0.64,0.644


In [5]:
df.columns

Index(['File Name', 'Peptide Sequence', 'Peptide Length', 'Cys Number',
       'Cys Content (# of Cys / Length)', 'Lys Number',
       'Lys Content (# of Lys / Length)', 'PTM Score', 'IPTM Score',
       'Confidence Score (0.2PTM + 0.8IPTM)'],
      dtype='object')

get the ones we order

In [6]:
all_oligos = pd.read_csv("A02_A01/patent_peptides_to_order.csv")
all_oligos.head(2)

,ID,Sequence
0,1,GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTC...
1,2,GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTC...


In [7]:
all_oligos = all_oligos["Sequence"].to_list()
len(all_oligos)

96

In [8]:
front = "GCGAATTAATACGACTCACTATAGGGAGACCACAACGGTTTCCCTCTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACATATGAAAATCGAAGAAGATTACAAGGATCACGATGGAGACTATAAGGATCACGACATCGATTACAAAGACGATGACGACAAATCAGGCCATCACCATCATCATCATCATCACTCGTCAGGC"
end = "TAGTAGTAAGGAGGATCCCGGGAATTCTCGAGTAAGGTTAACCTGCTAGCATAACCCCTTGGGGCCTCTAAACGGGTCTTGAGGGGTTTTTTG"

In [9]:
all_dna_pep = []
for x in all_oligos:
    dna = x.replace(front, "")
    dna = dna.replace(end, "")
    all_dna_pep.append(dna)
len(all_dna_pep)

96

In [10]:
from Bio.Seq import Seq

all_amino_acid_pep = []
for dna_seq in all_dna_pep:
    amino_acid_seq = Seq(dna_seq).translate()
    all_amino_acid_pep.append(str(amino_acid_seq))

len(all_amino_acid_pep)


96

get the score

In [11]:
path = "/Users/michaelmoret/Desktop/Externa/data/elisa/elisa_patented_peptides.xlsx"
df_score = pd.read_excel(path, sheet_name="Sheet1", header=None)
df_score.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.092,0.149,0.064,0.054,0.055,0.087,0.148,0.096,0.098,0.107,0.050,0.066
1,0.058,0.064,0.097,0.153,0.087,0.061,0.123,0.067,0.053,0.081,0.060,0.050
2,0.062,0.071,0.093,0.070,0.071,0.086,0.047,0.275,0.143,0.044,0.065,0.205


In [12]:
score_values = df_score.values.flatten().tolist()
len(score_values)

96

In [13]:
score_values[:4]

[0.092, 0.149, 0.064, 0.054]

In [14]:
score_values[9:14]

[0.107, 0.05, 0.066, 0.058, 0.064]

In [15]:
score_values[24:27]

[0.062, 0.071, 0.093]

get the computation score (in the right order)

In [16]:
all_comp_pep = df["Peptide Sequence"].to_list()
all_comp_score = df["Confidence Score (0.2PTM + 0.8IPTM)"].to_list()

d_cpep_cscore = dict(zip(all_comp_pep, all_comp_score))
len(d_cpep_cscore)

115

In [17]:
tested_comp_score = []
for pep in all_amino_acid_pep:
    tested_comp_score.append(d_cpep_cscore[pep])
len(tested_comp_score)

96

cleaning
- remove last three that are controls, kp, and long kp

In [18]:
tested_comp_score = tested_comp_score[:-3]
score_values = score_values[:-3]
len(tested_comp_score), len(score_values)

(93, 93)

remove the 0.685 which is a problem

In [19]:
index = score_values.index(0.685)
print(index)

84


In [20]:
score_values.pop(84)
tested_comp_score.pop(84)
len(tested_comp_score), len(score_values)

(92, 92)

In [21]:
correlation = np.corrcoef(tested_comp_score, score_values)[0, 1]
round(correlation, 3)

-0.134

explore

In [22]:
comp_explore = []
elisa_explore = []

for c,e in zip(tested_comp_score, score_values):
    if e<0.1 or e>2:
        comp_explore.append(c)
        elisa_explore.append(e)
len(comp_explore), len(elisa_explore)

(69, 69)

In [23]:
correlation = np.corrcoef(comp_explore, elisa_explore)[0, 1]
round(correlation, 3)

-0.345